In [1]:
import torch
import torch.nn as nn

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary
import torch.optim as optim
from vgg16 import _Vgg16, Block
from trainer.trainer_kfold import TrainerWithKFoldValidation

In [4]:
BATCH_SIZE = 32
EPOCH = 50

In [5]:
transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                              shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                           download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                             shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
               'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100.0%

Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
vgg_16_adam = _Vgg16(Block, [2, 2, 3, 3, 3])
optimizer_adam = optim.Adam(vgg_16_adam.parameters(), lr=0.001)

In [7]:
summary(vgg_16_adam, (3, 64, 64), device='cuda')

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 32, 32]          --
|    └─Block: 2-1                        [-1, 64, 64, 64]          --
|    |    └─Conv2d: 3-1                  [-1, 64, 64, 64]          1,792
|    |    └─BatchNorm2d: 3-2             [-1, 64, 64, 64]          128
|    |    └─ReLU: 3-3                    [-1, 64, 64, 64]          --
|    └─Block: 2-2                        [-1, 64, 64, 64]          --
|    |    └─Conv2d: 3-4                  [-1, 64, 64, 64]          36,928
|    |    └─BatchNorm2d: 3-5             [-1, 64, 64, 64]          128
|    |    └─ReLU: 3-6                    [-1, 64, 64, 64]          --
|    └─MaxPool2d: 2-3                    [-1, 64, 32, 32]          --
├─Sequential: 1-2                        [-1, 128, 16, 16]         --
|    └─Block: 2-4                        [-1, 128, 32, 32]         --
|    |    └─Conv2d: 3-7                  [-1, 128, 32, 32]         73,856
| 

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 32, 32]          --
|    └─Block: 2-1                        [-1, 64, 64, 64]          --
|    |    └─Conv2d: 3-1                  [-1, 64, 64, 64]          1,792
|    |    └─BatchNorm2d: 3-2             [-1, 64, 64, 64]          128
|    |    └─ReLU: 3-3                    [-1, 64, 64, 64]          --
|    └─Block: 2-2                        [-1, 64, 64, 64]          --
|    |    └─Conv2d: 3-4                  [-1, 64, 64, 64]          36,928
|    |    └─BatchNorm2d: 3-5             [-1, 64, 64, 64]          128
|    |    └─ReLU: 3-6                    [-1, 64, 64, 64]          --
|    └─MaxPool2d: 2-3                    [-1, 64, 32, 32]          --
├─Sequential: 1-2                        [-1, 128, 16, 16]         --
|    └─Block: 2-4                        [-1, 128, 32, 32]         --
|    |    └─Conv2d: 3-7                  [-1, 128, 32, 32]         73,856
| 

In [8]:
trainer = TrainerWithKFoldValidation(vgg_16_adam, optimizer_adam, trainset, testset, save_name="vgg16_adam_batch32_fold5", path=None, epoch=EPOCH, batch=BATCH_SIZE)

In [9]:
trainer.training(width=250)

[epoch=  1, batch_cnt=  250] loss: 2.27972
Train Acc: 18 %
Accuracy: 19 %
[epoch=  1, batch_cnt=  500] loss: 2.03966
Train Acc: 21 %
Accuracy: 21 %
[epoch=  1, batch_cnt=  750] loss: 1.95427
Train Acc: 22 %
Accuracy: 22 %
[epoch=  1, batch_cnt= 1000] loss: 1.86722
Train Acc: 25 %
Accuracy: 25 %
[epoch=  1, batch_cnt= 1250] loss: 1.82204
Train Acc: 27 %
Accuracy: 27 %
[epoch=  2, batch_cnt=  250] loss: 1.79648
Train Acc: 27 %
Accuracy: 27 %
[epoch=  2, batch_cnt=  500] loss: 1.74341
Train Acc: 30 %
Accuracy: 29 %
[epoch=  2, batch_cnt=  750] loss: 1.73984
Train Acc: 30 %
Accuracy: 29 %
[epoch=  2, batch_cnt= 1000] loss: 1.71296
Train Acc: 32 %
Accuracy: 32 %
[epoch=  2, batch_cnt= 1250] loss: 1.69031
Train Acc: 34 %
Accuracy: 33 %
[epoch=  3, batch_cnt=  250] loss: 1.64582
Train Acc: 37 %
Accuracy: 37 %
[epoch=  3, batch_cnt=  500] loss: 1.60390
Train Acc: 38 %
Accuracy: 38 %
[epoch=  3, batch_cnt=  750] loss: 1.55696
Train Acc: 39 %
Accuracy: 40 %
[epoch=  3, batch_cnt= 1000] loss: 1.5

Accuracy: 82 %
[epoch= 23, batch_cnt=  500] loss: 0.17185
Train Acc: 95 %
Accuracy: 82 %
[epoch= 23, batch_cnt=  750] loss: 0.15952
Train Acc: 96 %
Accuracy: 82 %
[epoch= 23, batch_cnt= 1000] loss: 0.17538
Train Acc: 96 %
Accuracy: 82 %
[epoch= 23, batch_cnt= 1250] loss: 0.16415
Train Acc: 95 %
Accuracy: 82 %
[epoch= 24, batch_cnt=  250] loss: 0.13792
Train Acc: 96 %
Accuracy: 82 %
[epoch= 24, batch_cnt=  500] loss: 0.14935
Train Acc: 95 %
Accuracy: 82 %
[epoch= 24, batch_cnt=  750] loss: 0.14844
Train Acc: 95 %
Accuracy: 82 %
[epoch= 24, batch_cnt= 1000] loss: 0.19800
Train Acc: 95 %
Accuracy: 82 %
[epoch= 24, batch_cnt= 1250] loss: 0.15791
Train Acc: 97 %
Accuracy: 83 %
[epoch= 25, batch_cnt=  250] loss: 0.20438
Train Acc: 93 %
Accuracy: 80 %
[epoch= 25, batch_cnt=  500] loss: 0.16291
Train Acc: 96 %
Accuracy: 82 %
[epoch= 25, batch_cnt=  750] loss: 0.11948
Train Acc: 97 %
Accuracy: 83 %
[epoch= 25, batch_cnt= 1000] loss: 0.14316
Train Acc: 96 %
Accuracy: 82 %
[epoch= 25, batch_cnt= 

Train Acc: 98 %
Accuracy: 83 %
[epoch= 45, batch_cnt=  750] loss: 0.06137
Train Acc: 98 %
Accuracy: 83 %
[epoch= 45, batch_cnt= 1000] loss: 0.03724
Train Acc: 99 %
Accuracy: 84 %
[epoch= 45, batch_cnt= 1250] loss: 0.06734
Train Acc: 98 %
Accuracy: 84 %
[epoch= 46, batch_cnt=  250] loss: 0.04079
Train Acc: 99 %
Accuracy: 83 %
[epoch= 46, batch_cnt=  500] loss: 0.05512
Train Acc: 98 %
Accuracy: 84 %
[epoch= 46, batch_cnt=  750] loss: 0.06070
Train Acc: 97 %
Accuracy: 82 %
[epoch= 46, batch_cnt= 1000] loss: 0.08520
Train Acc: 98 %
Accuracy: 84 %
[epoch= 46, batch_cnt= 1250] loss: 0.04966
Train Acc: 99 %
Accuracy: 84 %
[epoch= 47, batch_cnt=  250] loss: 0.03920
Train Acc: 98 %
Accuracy: 83 %
[epoch= 47, batch_cnt=  500] loss: 0.05994
Train Acc: 98 %
Accuracy: 83 %
[epoch= 47, batch_cnt=  750] loss: 0.04786
Train Acc: 99 %
Accuracy: 84 %
[epoch= 47, batch_cnt= 1000] loss: 0.09226
Train Acc: 98 %
Accuracy: 83 %
[epoch= 47, batch_cnt= 1250] loss: 0.05749
Train Acc: 98 %
Accuracy: 83 %
[epoch=

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [ ]:
import numpy as np

epoch_loss = np.array(trainer.epoch_loss).flatten()
epoch_acc = np.array(trainer.epoch_acc).flatten()
epoch_tacc = np.array(trainer.epoch_tacc).flatten()

epoch_loss = list(epoch_loss)
epoch_acc = list(epoch_acc)
epoch_tacc = list(epoch_tacc)

In [ ]:
import json
with open('epoch.json', 'w') as fp:
    epoch_info = {
        'acc': epoch_acc,
        'loss': epoch_loss,
        'tacc': epoch_tacc
    }
    json.dump(epoch_info, fp)

In [ ]:
checkpoint = {
    'model_stat': trainer.model.state_dict(),
    'optimizer_stat': trainer.optimizer.state_dict(),
    'loss': trainer.criterion.state_dict(),
}

torch.save(checkpoint, 'vgg16_adma_batch32.pth')